Sources:
- https://github.com/barbagroup/CFDPython/blob/master/lessons/04_Step_3.ipynb
- Derivation of the Heat Equation: Numerical Methods, Chapra

Difussion Equation in 1D (also known as the one dimensional heat diffusion equation)

$$\frac{\partial u}{\partial t}= \nu \frac{\partial^2 u}{\partial x^2}$$

where $u(x,t)$ is the unknown velocity function to be solved for, $x$ is a coordinate in space, and $t$ is time. The coefficient $\nu$ is the `kinematic viscocity` and determines how fast the fluid velocity $u$ changes in time. A quick short form for the difussion equation is $u_t = \nu u_{xx}$.

for heat transport problems

$$\frac{\partial u}{\partial t}= \alpha \frac{\partial^2 u}{\partial x^2}$$

the coefficient $\alpha$  is the `diffusion coefficient`

Typical diffusion problems may experience rapid change in the very beginning, but then the evolution of $u$ becomes slower and slower. The solution is usually very smooth, and after some time, one cannot recognize the initial shape of u. This is in sharp contrast to solutions of the wave equation where the initial shape is preserved - the solution is basically a moving initial condition. The one-dimensional linear convection equation has solutions that propagates with speed $c$ forever, without changing shape, while the diffusion equation converges to a stationary solution $\bar u(x)$ as $t\rightarrow\infty$. In this limit,  $\frac{\partial u}{\partial t} = 0$ and then $\bar u(x)$ is governed by $\frac{\partial^2 u}{\partial x^2} = 0$. This stationary limit of the diffusion equation is called the Laplace equation and arises in a very wide range of applications throughout the sciences.

It is possible to solve $u(x,t)$ using a explicit scheme, but the time step restrictions soon become much less favorable for an explicit scheme for the 1D linear convection equation. And for more importance, since the solution $u$ of the diffusion equation is very smooth and changes slowly, small-time steps are not convinient and not required by accuracy as the diffusion process converges to a stationary state.

$$\begin{split}\tag{3}
u(0,t)  = 0, \quad  t>0,\end{split}$$

$$\begin{split}\tag{4}
u(L,t)  = 0, \quad  t>0{\thinspace .}\end{split}$$